In [1]:

class Parent:
    def greet(self):
        return "Hello from Parent!"

class Child(Parent):
    def greet(self):
        return super().greet() + " And hello from Child!"

x = Child()
print(x.greet())  # Outputs: Hello from Parent! And hello from Child!

Hello from Parent! And hello from Child!


### Python program for Dijkstra's single source shortest path algorithm. 

In [4]:
class Path():

	def __init__(self, vertices):
		self.V = vertices
		self.graph = [[0 for column in range(vertices)]
					for row in range(vertices)]

	def printSolution(self, dist):
		print("Vertex \t Distance from Source")
		for node in range(self.V):
			print(node, "\t\t", dist[node])

	# A utility function to find the vertex with
	# minimum distance value, from the set of vertices
	# not yet included in shortest path tree
	def minDistance(self, dist, sptSet):

		# Initialize minimum distance for next node
		min = 1e7

		# Search not nearest vertex not in the
		# shortest path tree
		for v in range(self.V):
			if dist[v] < min and sptSet[v] == False:
				min = dist[v]
				min_index = v

		return min_index

	# Function that implements Dijkstra's single source
	# shortest path algorithm for a graph represented
	# using adjacency matrix representation
	def dijkstra(self, src):

		dist = [1e7] * self.V
		dist[src] = 0
		sptSet = [False] * self.V

		for cout in range(self.V):

			# Pick the minimum distance vertex from
			# the set of vertices not yet processed.
			# u is always equal to src in first iteration
			u = self.minDistance(dist, sptSet)

			# Put the minimum distance vertex in the
			# shortest path tree
			sptSet[u] = True

			# Update dist value of the adjacent vertices
			# of the picked vertex only if the current
			# distance is greater than new distance and
			# the vertex in not in the shortest path tree
			for v in range(self.V):
				if (self.graph[u][v] > 0 and
				sptSet[v] == False and
				dist[v] > dist[u] + self.graph[u][v]):
					dist[v] = dist[u] + self.graph[u][v]

		self.printSolution(dist)

### The program is for adjacency matrix representation of the graph

In [5]:
# Driver program
g = Path(9)
g.graph = [[0, 4, 0, 0, 0, 0, 0, 8, 0],
		[4, 0, 8, 0, 0, 0, 0, 11, 0],
		[0, 8, 0, 7, 0, 4, 0, 0, 2],
		[0, 0, 7, 0, 9, 14, 0, 0, 0],
		[0, 0, 0, 9, 0, 10, 0, 0, 0],
		[0, 0, 4, 14, 10, 0, 2, 0, 0],
		[0, 0, 0, 0, 0, 2, 0, 1, 6],
		[8, 11, 0, 0, 0, 0, 1, 0, 7],
		[0, 0, 2, 0, 0, 0, 6, 7, 0]
		]

g.dijkstra(0)


Vertex 	 Distance from Source
0 		 0
1 		 4
2 		 12
3 		 19
4 		 21
5 		 11
6 		 9
7 		 8
8 		 14


In [5]:
import tkinter as tk
import math
from heapq import heappop, heappush
import random
from gtts import gTTS
import os
import pygame
import time

# Define the centers of each table
table_locations = [(200, 200), (600, 200), (200, 600), (600, 600), (1000, 200), (1400, 200), (1000, 600), (1400, 600)]

table_points = []
destination_points = []

COMMAND_FULL = ""

active_location = (200,320)

def text_to_speech(text, lang='en'):
    tts = gTTS(text=text, lang=lang)
    tts.save("C:/Users/yasir/Downloads/output.mp3")
    pygame.init()
    pygame.mixer.init()
    pygame.mixer.music.load("C:/Users/yasir/Downloads/output.mp3")
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        pygame.time.Clock().tick(10)
    pygame.mixer.music.stop()  # Stop the music
    pygame.mixer.quit()  # Unload the music

def play_welcome_phrase():
    welcome_text = "Welcome to the restaurant. Please select a table. To deliver the order"
    text_to_speech(" Hi ")
    text_to_speech(welcome_text)


def table_clicked(x, y, canvas):
    global COMMAND_FULL
    global active_location
    print("Clicked at", x, y)
    print("Table", str(active_location), "is active")
    print("Table",str(active_location), "coordinates:", active_location)

    canvas.create_oval(active_location[0] - 6, active_location[1] - 6, active_location[0] + 6, active_location[1] + 6, fill="cyan")

    # Calculate shortest path
    path = find_shortest_path((x, y), (active_location[0], active_location[1]))
    print("Shortest path:", path)
    draw_path(path, canvas)  # Pass canvas to draw_path
    COMMAND = ""
    if len(path) == 2:
        COMMAND = COMMAND + str(abs(path[0][0] - path[1][0])) + " "
    else:
        COMMAND = COMMAND + str(abs(path[0][0] - path[1][0])) + " "
        if path[0][0] < path[2][0]:
            if path[1][1] < path[2][1]:
                COMMAND = COMMAND + str("R") + " "
            else:
                COMMAND = COMMAND + str("L") + " "
        else:
            if path[1][1] < path[2][1]:
                COMMAND = COMMAND + str("L") + " "
            else:
                COMMAND = COMMAND + str("R") + " "
        COMMAND = COMMAND + str(abs(path[1][1] - path[2][1])) + " "
        if path[1][1] < path[3][1]:
            if path[2][0] < path[3][0]:
                COMMAND = COMMAND + str("L") + " "
            else:
                COMMAND = COMMAND + str("R") + " "
        else:
            if path[2][0] < path[3][0]:
                COMMAND = COMMAND + str("R") + " "
            else:
                COMMAND = COMMAND + str("L") + " "
        COMMAND = COMMAND + str(abs(path[2][0] - path[3][0])) + " "

    print("Command:", COMMAND)
    COMMAND_FULL = "COMMAND: " + COMMAND

    canvas.create_rectangle(0,20,1600,60, fill="white")
    canvas.create_text(800,40, text = COMMAND_FULL, font=("Arial", 20, "bold"), fill="#000000")

    text_to_speech(COMMAND_FULL)
    
    active_location = [x,y]

def heuristic(a, b):
    return math.sqrt((b[0] - a[0])**2 + (b[1] - a[1])**2)

def find_shortest_path(goal, start):
   path = []
   path.append(start)
   up_down = True
   if start[1] == goal[1]:
       up_down = False

   left_pane = False

   middle_pane = (start[0] + goal[0])//2

   if goal[0] < middle_pane:
       left_pane = True

   if up_down:
       if not left_pane:
           path.append((goal[0] - 200, start[1]))
           path.append((goal[0] - 200, goal[1]))
           path.append(goal)
       else:
           path.append((goal[0] + 200, start[1]))
           path.append((goal[0] + 200, goal[1]))
           path.append(goal)
   else:
         path.append((goal[0], goal[1]))
   
   return path

def draw_path(path, canvas):  # Pass canvas to draw_path
    path_color = random.choice(["red", "green", "blue", "yellow", "purple", "orange", "cyan", "magenta", "brown", "pink"])
    if path:
        for i in range(len(path) - 1):
            x1, y1 = path[i]
            x2, y2 = path[i + 1]
            canvas.create_line(x1, y1, x2, y2, fill=path_color, width=4)
            time.sleep(1)

def main():
    global table_points, destination_points, COMMAND_FULL
    root = tk.Tk()
    root.title("Restaurant Tables")

    canvas = tk.Canvas(root, width=1600, height=800)
    canvas.pack()

    # Draw the grid
    for x in range(0, 1600, 20):
        canvas.create_line(x, 0, x, 1600, fill="lightgray", tags="grid")
    for y in range(0, 800, 20):
        canvas.create_line(0, y, 1600, y, fill="lightgray", tags="grid")

    table_points = []  # Clear the table points array
    destination_points = []  # Clear the destination points array

    # Use table_locations array to define the coordinates and size of each table
    for i, (center_x, center_y) in enumerate(table_locations, start=1):
        table_size = 200  # Define the size of each table
        x1 = center_x - table_size // 2
        y1 = center_y - table_size // 2
        x2 = center_x + table_size // 2
        y2 = center_y + table_size // 2
        table = canvas.create_rectangle(x1, y1, x2, y2, fill="#00FF80")
        canvas.create_text(center_x, center_y, text="Table " + str(i), font=("", 20, "bold"), fill="#000000")

        # Draw blue dot under 1 grid point from each table
        dot_x = center_x
        dot_y = center_y + table_size // 2 + 20  # Offset by one grid point downwards
        if i == active_location:
            dot_color = "cyan"
        else:
            dot_color = "blue"
        canvas.create_oval(dot_x - 6, dot_y - 6, dot_x + 6, dot_y + 6, fill=dot_color)
        destination_points.append((dot_x, dot_y))

        # Binding the click event to each table
        canvas.tag_bind(table, "<Button-1>", lambda event, x=center_x, y=center_y: table_clicked(x, y + table_size // 2 + 20, canvas))

        # Calculate and add points inside the table to table_points array
        for x in range(x1, x2+20, 20):
            for y in range(y1, y2 + 20, 20):
                table_points.append((x, y))

        for point in table_points:
            canvas.create_rectangle(point[0]-1, point[1]-1, point[0] + 1, point[1] + 1, fill="black")

    root.after(100, play_welcome_phrase)

    root.mainloop()

if __name__ == "__main__":
    main()


Clicked at 1400 320
Table (200, 320) is active
Table (200, 320) coordinates: (200, 320)
Shortest path: [(200, 320), (1400, 320)]
Command: 1200 
Clicked at 1400 320
Table (200, 320) is active
Table (200, 320) coordinates: (200, 320)
Shortest path: [(200, 320), (1400, 320)]


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\hbgal\miniconda3\envs\py310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\hbgal\AppData\Local\Temp\ipykernel_33144\3005658280.py", line 167, in <lambda>
    canvas.tag_bind(table, "<Button-1>", lambda event, x=center_x, y=center_y: table_clicked(x, y + table_size // 2 + 20, canvas))
  File "C:\Users\hbgal\AppData\Local\Temp\ipykernel_33144\3005658280.py", line 85, in table_clicked
    text_to_speech(COMMAND_FULL)
  File "C:\Users\hbgal\AppData\Local\Temp\ipykernel_33144\3005658280.py", line 22, in text_to_speech
    tts.save("C:/Users/yasir/Downloads/output.mp3")
  File "c:\Users\hbgal\miniconda3\envs\py310\lib\site-packages\gtts\tts.py", line 334, in save
    with open(str(savefile), "wb") as f:
FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/yasir/Downloads/output.mp3'
Exception in Tkinter callback
Traceback (most recent call last):
  Fil

Command: 1200 
Clicked at 1400 320
Table (200, 320) is active
Table (200, 320) coordinates: (200, 320)
Shortest path: [(200, 320), (1400, 320)]


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\hbgal\miniconda3\envs\py310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\hbgal\AppData\Local\Temp\ipykernel_33144\3005658280.py", line 167, in <lambda>
    canvas.tag_bind(table, "<Button-1>", lambda event, x=center_x, y=center_y: table_clicked(x, y + table_size // 2 + 20, canvas))
  File "C:\Users\hbgal\AppData\Local\Temp\ipykernel_33144\3005658280.py", line 85, in table_clicked
    text_to_speech(COMMAND_FULL)
  File "C:\Users\hbgal\AppData\Local\Temp\ipykernel_33144\3005658280.py", line 22, in text_to_speech
    tts.save("C:/Users/yasir/Downloads/output.mp3")
  File "c:\Users\hbgal\miniconda3\envs\py310\lib\site-packages\gtts\tts.py", line 334, in save
    with open(str(savefile), "wb") as f:
FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/yasir/Downloads/output.mp3'


Command: 1200 


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\hbgal\miniconda3\envs\py310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\hbgal\AppData\Local\Temp\ipykernel_33144\3005658280.py", line 167, in <lambda>
    canvas.tag_bind(table, "<Button-1>", lambda event, x=center_x, y=center_y: table_clicked(x, y + table_size // 2 + 20, canvas))
  File "C:\Users\hbgal\AppData\Local\Temp\ipykernel_33144\3005658280.py", line 85, in table_clicked
    text_to_speech(COMMAND_FULL)
  File "C:\Users\hbgal\AppData\Local\Temp\ipykernel_33144\3005658280.py", line 22, in text_to_speech
    tts.save("C:/Users/yasir/Downloads/output.mp3")
  File "c:\Users\hbgal\miniconda3\envs\py310\lib\site-packages\gtts\tts.py", line 334, in save
    with open(str(savefile), "wb") as f:
FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/yasir/Downloads/output.mp3'
